# Metrics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

In [ ]:
def to_np_array(array):
    # Keep None as it is, and convert others into Numpy array
    if array is not None and not isinstance(array, np.ndarray):
        array = np.array(array)
    if array is not None:
        array = array.squeeze()
    return array

In [ ]:
class Metric:

    def __init__(self, plot_name, split):
        self.plot_name = plot_name
        self.split = split

    def __call__(self, y_true, y_pred=None, p_pred=None, plot = False):
        y_true = to_np_array(y_true)
        y_pred = to_np_array(y_pred)
        p_pred = to_np_array(p_pred)
        return self.forward(y_true, y_pred, p_pred, plot)

    def forward(self, y_true, y_pred=None, p_pred=None, plot = False):
        raise NotImplementedError("This is the base class for metrics")

In [ ]:
class CorrAndRSquared(Metric):

  def forward(self, y_true, y_pred = None, p_pred = None, plot = False):

    r2 = r2_score(y_true, y_pred)
    pearson, pvalue = pearsonr(y_true, y_pred)
    if plot:
      self._plot_scatter(y_true, y_pred, r2, pearson, pvalue)

    return r2, (pearson, pvalue)

  def _plot_scatter(self, y_true, y_pred, r2, pearson, pvalue):

    plt.rcParams['axes.labelweight'] = 'bold'
    plt.rcParams['axes.titleweight'] = 'bold'

    plt.figure(figsize = (8, 6))

    plt.scatter(y_pred, y_true, color = 'black', label = 'Actual Performance')
    plt.plot(y_true, y_true, color = 'red', linestyle = '--', label = 'Theoretical Perfect Performance')

    plt.xlabel('Predicted Value', size = 12)
    plt.ylabel('True Value', size = 12)
    plt.title('True Values vs Predicted Values', size = 13)

    plt.text(110, 15, f'R-Squared = {r2:.2f}', fontweight = 'bold')
    if pvalue < 0.001:
      plt.text(110, 5, f'Pearson Correlation = {pearson:.2f} (p < 0.001)', fontweight = 'bold')
    else:
      plt.text(110, 5, f'Pearson Correlation = {pearson:.2f} (p = {pvalue:.4f})', fontweight = 'bold')

    plt.legend(loc = 'upper left')

    plt.show()

    plt.savefig('/content/drive/MyDrive/BINF_4008_Final_Project/Plots/' + self.plot_name + '_' + self.split + '_r2.png')

In [ ]:
class MSE(Metric):

  def forward(self, y_true, y_pred = None, p_pred = None, plot = False):

    mse = mean_squared_error(y_true, y_pred)

    return mse

In [ ]:
class MAE(Metric):

  def forward(self, y_true, y_pred = None, p_pred = None, plot = False):

    mae = mean_absolute_error(y_true, y_pred)

    return mae

In [ ]:
class MAPE(Metric):

  def forward(self, y_true, y_pred = None, p_pred = None, plot = False):

    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return mape

In [ ]:
class PlotLoss:

  def __init__(self, plot_name):

    self.plot_name = plot_name

  def __call__(self, train_losses, val_losses):

    train_losses = to_np_array(train_losses)
    val_losses = to_np_array(val_losses)

    self.forward(train_losses, val_losses, self.plot_name)

  def forward(self, train_losses, val_losses, plot_name):

    plt.figure(figsize = (6, 4))

    plt.plot(range(len(train_losses)), train_losses, lw = 2, color = 'navy', label = 'Train Loss')

    plt.plot(range(len(val_losses)), val_losses, lw = 2, color = 'darkorange', label = 'Val Loss')

    plt.title(f'Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc = 'lower right')

    plt.xticks(ticks = range(len(train_losses)), labels = range(1, len(train_losses) + 1))

    plt.show()

    plt.savefig('/content/drive/MyDrive/BINF_4008_Final_Project/Plots/' + plot_name + '_loss.png')

In [ ]:
class PlotMetrics:

  def __init__(self, plot_name):
    self.plot_name = plot_name

  def __call__(self, train_metric, val_metric):

    train_metric = to_np_array(train_metric)
    val_metric = to_np_array(val_metric)

    self.forward(train_metric, val_metric, self.plot_name)

  def forward(self, train_metric, val_metric, plot_name):
    print('This is the base for plotting.')

In [ ]:
class PlotR2(PlotMetrics):

  def forward(self, train_metric, val_metric, plot_name):

    plt.figure(figsize = (6, 4))

    plt.plot(range(len(train_metric)), train_metric, lw = 2, color = 'navy', label = 'Train R2')

    plt.plot(range(len(val_metric)), val_metric, lw = 2, color = 'darkorange', label = 'Val R2')

    plt.title(f'R2 Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('R2')

    plt.legend(loc = 'lower right')

    plt.xticks(ticks = range(len(train_metric)), labels = range(1, len(train_metric) + 1))

    plt.show()

    plt.savefig('/content/drive/MyDrive/BINF_4008_Final_Project/Plots/' + self.plot_name + '_R2_over_epochs.png')

In [ ]:
class PlotPearson(PlotMetrics):

  def forward(self, train_metric, val_metric, plot_name):

    plt.figure(figsize = (6, 4))

    plt.plot(range(len(train_metric)), train_metric, lw = 2, color = 'navy', label = 'Train Pearson')

    plt.plot(range(len(val_metric)), val_metric, lw = 2, color = 'darkorange', label = 'Val Pearson')

    plt.title(f'Pearson Coefficient Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Pearson Coefficient')

    plt.legend(loc = 'lower right')

    plt.xticks(ticks = range(len(train_metric)), labels = range(1, len(train_metric) + 1))

    plt.show()

    plt.savefig('/content/drive/MyDrive/BINF_4008_Final_Project/Plots/' + self.plot_name + '_pearson_over_epochs.png')

In [ ]:
class PlotMSE(PlotMetrics):

  def forward(self, train_metric, val_metric, plot_name):

    plt.figure(figsize = (6, 4))

    plt.plot(range(len(train_metric)), train_metric, lw = 2, color = 'navy', label = 'Train MSE')

    plt.plot(range(len(val_metric)), val_metric, lw = 2, color = 'darkorange', label = 'Val MSE')

    plt.title(f'MSE Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('MSE')

    plt.legend(loc = 'lower right')

    plt.xticks(ticks = range(len(train_metric)), labels = range(1, len(train_metric) + 1))

    plt.show()

    plt.savefig('/content/drive/MyDrive/BINF_4008_Final_Project/Plots/' + self.plot_name + '_MSE_over_epochs.png')

In [ ]:
class PlotMAE(PlotMetrics):

  def forward(self, train_metric, val_metric, plot_name):

    plt.figure(figsize = (6, 4))

    plt.plot(range(len(train_metric)), train_metric, lw = 2, color = 'navy', label = 'Train MAE')

    plt.plot(range(len(val_metric)), val_metric, lw = 2, color = 'darkorange', label = 'Val MAE')

    plt.title(f'MAE Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')

    plt.legend(loc = 'lower right')

    plt.xticks(ticks = range(len(train_metric)), labels = range(1, len(train_metric) + 1))

    plt.show()

    plt.savefig('/content/drive/MyDrive/BINF_4008_Final_Project/Plots/' + self.plot_name + '_MAE_over_epochs.png')

In [ ]:
class PlotMAPE(PlotMetrics):

  def forward(self, train_metric, val_metric, plot_name):

    plt.figure(figsize = (6, 4))

    plt.plot(range(len(train_metric)), train_metric, lw = 2, color = 'navy', label = 'Train MAPE')

    plt.plot(range(len(val_metric)), val_metric, lw = 2, color = 'darkorange', label = 'Val MAPE')

    plt.title(f'MAPE Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('MAPE')

    plt.legend(loc = 'lower right')

    plt.xticks(ticks = range(len(train_metric)), labels = range(1, len(train_metric) + 1))

    plt.show()

    plt.savefig('/content/drive/MyDrive/BINF_4008_Final_Project/Plots/' + self.plot_name + '_MAPE_over_epochs.png')